# Field Basics

In [1]:
using CMBLensing

## Base Fields

The basic building blocks of CMBLensing.jl are CMB "fields", like temperature, Q or U polarization, or the lensing potential $\phi$. These types are all encompassed by the abstract type `Field`, with some concrete examples including `FlatMap` for a flat-sky map projection, or `FlatQUMap` for Q/U polarization, etc...

`Flat` fields are just thin wrappers around Julia arrays, e.g.

In [2]:
Ix = rand(2,2)

2×2 Array{Float64,2}:
 0.718554   0.783272
 0.0796566  0.927715

In [3]:
f = FlatMap(Ix)

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.7185537948681697 
 0.07965658845377166
 0.783271558132157  
 0.9277154735380295 

When displayed, you can see the pixels in the 2x2 map have been splayed out into a length-4 array. This is intentional, as even though the maps themselves are two-dimensional, it is extremely useful conceptually to think of fields as vectors (which they are, in fact, as they form an [abstract vector space](https://en.wikipedia.org/wiki/Vector_space)). This tie to vector spaces is deeply rooted in CMBLensing, to the extent that `Field` objects are a subtype of Julia's own `AbstractVector` type, 

In [4]:
f isa AbstractVector

true

The data itself, however, is still stored as the original 2x2 matrix, and can be accessed as follows,

In [5]:
f.Ix

2×2 Array{Float64,2}:
 0.718554   0.783272
 0.0796566  0.927715

But since `Fields` are vectors, they can be tranposed,

In [6]:
f'

1×4 LinearAlgebra.Adjoint{Float64,FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}}:
 0.718554  0.0796566  0.783272  0.927715

inner products can be computed,

In [7]:
f' * f

1.689640956312564e-7

and they can be added with each other as well as multiplied by scalars,

In [8]:
2*f+f

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 2.1556613846045094 
 0.23896976536131498
 2.349814674396471  
 2.783146420614089  

## Diagonal operators

Vector spaces have linear operators which act on the vectors. Linear operators correpsond to matrices, thus for a map with $N$ total pixels, a general linear operator would be an $N$-by-$N$ matrix, which for even modest map sizes becomes far too large to actually store. Thus, an important class of linear operators are ones which are diagonal, since these can actually be stored. CMBLensing uses Julia's builtin `Diagonal` to represent these. `Diagonal(f)` takes a vector `f` and puts it on the diagonal of the matrix:

In [9]:
Diagonal(f)

4×4 Diagonal{Float64,FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}}:
 0.718554   ⋅          ⋅         ⋅      
  ⋅        0.0796566   ⋅         ⋅      
  ⋅         ⋅         0.783272   ⋅      
  ⋅         ⋅          ⋅        0.927715

Multiplying this operator by the original map is then a matrix-vector product:

In [10]:
Diagonal(f) * f

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.5163195561194477  
 0.006345172084093549
 0.613514333778777   
 0.8606559998418903  

Note that this is also equal to the the pointwise multiplication of `f` with itself:

In [11]:
f .* f

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.5163195561194477  
 0.006345172084093549
 0.613514333778777   
 0.8606559998418903  

## Field Tuples

You can put `Fields` together into tuples. For example, 

In [12]:
a = FlatMap(rand(2,2))
b = FlatMap(rand(2,2));

In [13]:
FieldTuple(a,b)

8-element Field2Tuple{CMBLensing.BasisTuple{Tuple{Map,Map}}, Float64}:
 0.983644462883015    
 0.0045112141819265705
 0.7624669319673323   
 0.42287235402564716  
 0.012913662988633723 
 0.575161408331945    
 0.9879403809696659   
 0.7462386479019307   

The components can also have names:

In [14]:
ft = FieldTuple(a=a, b=b)

8-element Field2Tuple{(:a, :b), CMBLensing.BasisTuple{Tuple{Map,Map}}, Float64}:
 0.983644462883015    
 0.0045112141819265705
 0.7624669319673323   
 0.42287235402564716  
 0.012913662988633723 
 0.575161408331945    
 0.9879403809696659   
 0.7462386479019307   

which can be accessed later:

In [15]:
ft.a

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.983644462883015    
 0.0045112141819265705
 0.7624669319673323   
 0.42287235402564716  

`FieldTuples` have all of the same behavior of individual fields. Indeed, spin fields like QU or IQU are simply special `FieldTuples`:

In [16]:
fqu = FlatQUMap(a,b)
fqu isa FieldTuple

true

## Field Vectors

*in progress*

## Basis Conversion

All fields are tagged as to which basis they are stored in. You can convert them to other bases by calling the basis type on them:

In [17]:
f

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.7185537948681697 
 0.07965658845377166
 0.783271558132157  
 0.9277154735380295 

In [18]:
f′ = Fourier(f)

4-element FlatFourier{2×2 map, 1′ pixels, fourier∂, Array{Complex{Float64}}}:
   2.509197414992128 + 0.0im
 0.49445329100852553 + 0.0im
 -0.9127766483482451 + 0.0im
  0.7833411218202706 + 0.0im

Basis conversion is usually done automatically for you. E.g. here `f′` is automatically converted to a `FlatMap` before addition:

In [19]:
f + f′

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 1.4371075897363395 
 0.15931317690754337
 1.566543116264314  
 1.855430947076059  

A key feature of `Diagonal` operators is they convert the field they are acting on to the right basis before multiplication:

In [20]:
Diagonal(f) * f′

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.5163195561194477  
 0.006345172084093553
 0.6135143337787771  
 0.8606559998418905  

A `FlatMap` times a `FlatFourier` doesn't have a natural linear algebra meaning so its an error:

In [21]:
f * f′

MethodError: MethodError: no method matching *(::FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}, ::FlatFourier{2×2 map, 1′ pixels, fourier∂, Array{Complex{Float64}}})
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529
  *(!Matched::FuncOp, ::Field) at /home/marius/work/baylens/src/specialops.jl:116
  *(!Matched::NoLensing, ::Field) at /home/marius/work/baylens/src/lensing.jl:62
  ...

## Properties and indices

`FlatMap` and `FlatFourier` can be indexed directly like arrays. If given 1D indices, this is the index into the vector representation:

In [22]:
f

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.7185537948681697 
 0.07965658845377166
 0.783271558132157  
 0.9277154735380295 

In [23]:
f[1], f[2], f[3], f[4]

(0.7185537948681697, 0.07965658845377166, 0.783271558132157, 0.9277154735380295)

In [24]:
f[5]

BoundsError: BoundsError: attempt to access 2×2 Array{Float64,2} at index [5]

Or with a 2D index, this indexes directly into the 2D map:

In [25]:
f[1,1], f[2,1], f[1,2], f[2,2]

(0.7185537948681697, 0.07965658845377166, 0.783271558132157, 0.9277154735380295)

*Note:* there is no overhead to indexing `f` in this way as compared to working directly on the underlying array.

For other fields which are built on `FieldTuples`, 1D indexing will instead index the tuple indices:

In [26]:
ft

8-element Field2Tuple{(:a, :b), CMBLensing.BasisTuple{Tuple{Map,Map}}, Float64}:
 0.983644462883015    
 0.0045112141819265705
 0.7624669319673323   
 0.42287235402564716  
 0.012913662988633723 
 0.575161408331945    
 0.9879403809696659   
 0.7462386479019307   

In [27]:
ft[1]

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.983644462883015    
 0.0045112141819265705
 0.7624669319673323   
 0.42287235402564716  

In [28]:
ft[2]

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.012913662988633723
 0.575161408331945   
 0.9879403809696659  
 0.7462386479019307  

In [29]:
ft[3]

BoundsError: BoundsError: attempt to access NamedTuple{(:a, :b),Tuple{FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}},FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}}}
  at index [3]

To get the underlying data arrays, use the object's properties:

In [30]:
f.Ix

2×2 Array{Float64,2}:
 0.718554   0.783272
 0.0796566  0.927715

You can always find out what properties are available by typing `f.<Tab>`. For example, if you typed `ft` then hit `<Tab>` you'd get:

In [31]:
ft |> propertynames

(:fs, :a, :b)

For a `FieldTuple` like the `FlatQUMap` object, `fqu`, you can get each individual Q or U field:

In [32]:
fqu.Q

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.983644462883015    
 0.0045112141819265705
 0.7624669319673323   
 0.42287235402564716  

Or `fqu.Qx` which is shorthand for `fqu.Q.Ix`:

In [33]:
fqu.Q.Ix === fqu.Qx

true

If you convert `f` to Fourier space, it would have the `Il` property to get the Fourier coefficients of the $I$ component:

In [34]:
Fourier(f).Il

2×2 Array{Complex{Float64},2}:
   2.5092+0.0im  -0.912777+0.0im
 0.494453+0.0im   0.783341+0.0im

For convenience, you can index fields with brackets `[]` and any necessary conversions will be done automatically:

In [35]:
f[:Il]

2×2 Array{Complex{Float64},2}:
   2.5092+0.0im  -0.912777+0.0im
 0.494453+0.0im   0.783341+0.0im

This works between any bases. For example. `fqu` is originally `QUMap` but we can convert to `EBFourier` and get the `El` coefficients:

In [36]:
fqu[:El]

2×2 Array{Complex{Float64},2}:
 -2.17349-0.0im  0.197184+0.0im
  1.31873+0.0im  0.803949+0.0im

The general rule to keep in mind for these two ways of accessing the underlying data is:

* **Properties** (i.e. `f.Ix`) are type-stable and get you the underlying data arrays, even recursively from special `FieldTuples` like `FlatQUMap`, etc... If these arrays are modified, they affect the original field.
* **Indices** (i.e. `f[:Ix]`) are not type-stable, and may or may not be one of the underlying data arrays (because a basis conversion may have been performed). They should be used for getting (not setting) data, and in non-performance-critical code. 